In [4]:
import sys 
import numpy as np
import pandas as pd
import statsmodels as sm
import sklearn
import scipy as sp
%matplotlib inline 
import matplotlib.pyplot as plt
import math

In [5]:
df = pd.read_csv('dataframe_train.csv')
df

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,date,group,level,speed,...,source_type,source_tracking_id,source_lng,source_lat,target_lng,target_lat,grid_distance,expected_use_time,urgency,hour
0,10007871,0,2.100070e+18,121.630997,39.142343,PICKUP,20200201,2.020020e+16,3,4.751832,...,ASSIGN,2.100070e+18,121.630997,39.142343,121.632547,39.141946,377.0,804,1246,11
1,10007871,0,2.100070e+18,121.630997,39.142343,DELIVERY,20200201,2.020020e+16,3,4.751832,...,PICKUP,2.100070e+18,121.632547,39.141946,121.626144,39.140281,780.0,298,1246,11
2,10007871,0,2.100070e+18,121.630997,39.142343,PICKUP,20200201,2.020020e+16,3,4.751832,...,DELIVERY,2.100070e+18,121.626144,39.140281,121.631219,39.141811,550.0,545,2462,11
3,10007871,0,2.100070e+18,121.630997,39.142343,DELIVERY,20200201,2.020020e+16,3,4.751832,...,PICKUP,2.100070e+18,121.631219,39.141811,121.632084,39.146201,707.0,341,1205,11
4,10007871,0,2.100070e+18,121.630997,39.142343,PICKUP,20200201,2.020020e+16,3,4.751832,...,DELIVERY,2.100070e+18,121.632084,39.146201,121.631574,39.142231,770.0,166,1882,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509599,125996858,0,2.100080e+18,121.453188,39.208522,DELIVERY,20200227,2.020020e+17,3,5.573470,...,DELIVERY,2.100080e+18,121.445284,39.211421,121.444934,39.211511,1.0,2,2300,12
509600,125996858,0,2.100080e+18,121.453188,39.208522,PICKUP,20200227,2.020020e+17,3,5.573470,...,DELIVERY,2.100080e+18,121.444934,39.211511,121.447360,39.211273,476.0,672,2306,12
509601,125996858,0,2.100080e+18,121.453188,39.208522,PICKUP,20200227,2.020020e+17,3,5.573470,...,PICKUP,2.100080e+18,121.447360,39.211273,121.447360,39.211273,1.0,2,563,12
509602,125996858,0,2.100080e+18,121.453188,39.208522,DELIVERY,20200227,2.020020e+17,3,5.573470,...,PICKUP,2.100080e+18,121.447360,39.211273,121.453554,39.208541,1255.0,333,988,12


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509604 entries, 0 to 509603
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   courier_id              509604 non-null  int64  
 1   wave_index              509604 non-null  int64  
 2   tracking_id             509604 non-null  float64
 3   courier_wave_start_lng  509604 non-null  float64
 4   courier_wave_start_lat  509604 non-null  float64
 5   action_type             509604 non-null  object 
 6   date                    509604 non-null  int64  
 7   group                   509604 non-null  float64
 8   level                   509604 non-null  int64  
 9   speed                   509604 non-null  float64
 10  max_load                509604 non-null  int64  
 11  weather_grade           509604 non-null  object 
 12  aoi_id                  509604 non-null  object 
 13  shop_id                 509604 non-null  object 
 14  id                  

In [7]:
df_train = df[['speed', 'max_load', 'weather_grade', 'source_type', 'grid_distance', 
                     'urgency', 'hour']]
df_train = pd.get_dummies(df_train)
for i in df_train['hour'].unique():
    df_train[str(i)] = (df_train['hour'] == i).astype(int)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509604 entries, 0 to 509603
Data columns (total 30 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   speed                               509604 non-null  float64
 1   max_load                            509604 non-null  int64  
 2   grid_distance                       509604 non-null  float64
 3   urgency                             509604 non-null  int64  
 4   hour                                509604 non-null  int64  
 5   weather_grade_Bad Weather           509604 non-null  uint8  
 6   weather_grade_Normal Weather        509604 non-null  uint8  
 7   weather_grade_Slightly Bad Weather  509604 non-null  uint8  
 8   weather_grade_Very Bad Weather      509604 non-null  uint8  
 9   source_type_ASSIGN                  509604 non-null  uint8  
 10  source_type_DELIVERY                509604 non-null  uint8  
 11  source_type_PICKUP        

In [7]:
#linear regression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

x = df_train
y = df['expected_use_time'].ravel()
param_grid = [{'alpha':np.arange(0.01,0.51,0.01)}]

Lasso_LinReg = ElasticNet(l1_ratio = 1)
grid_1 = GridSearchCV(Lasso_LinReg, param_grid,cv=5)
grid_1.fit(x,y)
print("The best alpha parameter is",grid_1.best_params_) 
print('\n') 
print("The corresponding R-squared is %0.4f." % grid_1.best_score_) 

/Users/serena/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2866729532.9059143, tolerance: 6876140.20890488
  model = cd_fast.enet_coordinate_descent(


The best alpha parameter is {'alpha': 0.02}


The corresponding R-squared is 0.2509.


In [8]:
#Decision Tree
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
param_grid = {"max_depth": [2, 3, 4],
              "ccp_alpha": np.arange(0, 0.01, 0.001).tolist(),
              "min_samples_split": [10, 20, 40],
              "min_samples_leaf": [20, 40, 60],
              "max_leaf_nodes": [5, 20, 80]}
              
grid_cv_tree = GridSearchCV(tree, param_grid, scoring = 'r2', cv = 5)
grid_cv_tree.fit(x, y)

print("The best parameters are ",grid_cv_tree.best_params_) 
print() 
print("The score is %0.4f." % grid_cv_tree.best_score_)

The best parameters are  {'ccp_alpha': 0.0, 'max_depth': 4, 'max_leaf_nodes': 20, 'min_samples_leaf': 60, 'min_samples_split': 10}

The score is 0.3518.


In [14]:
#XGBT Model
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
data_train = xgb.DMatrix(data=x,label= y)
data_test = xgb.DMatrix(data=x,label= y)

usetime_xgbt = xgb.XGBRegressor(use_label_encoder=False)

param_grid = [{
    'n_estimators': [10, 30, 50, 70, 100],
    'learning_rate': [0.1, 0.3, 0.5, 0.7],
    'max_depth': range(1, 5),
    'gamma': [0,0.01,0.02,0.03],
    }]

grid2 = GridSearchCV(usetime_xgbt, param_grid, scoring ='r2', cv = 5)
model = grid2.fit(x, y)

print("The best parameters are ",model.best_params_) 
print() 
print("The best score for the XGBT is %0.4f." % model.best_score_)


The best parameters are  {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}

The best score for the XGBT is 0.3812.


In [15]:
#Compute OOS score of the best model so far -- XGBT Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split as tr_te_split
x = df_train
y = df['expected_use_time'].ravel()
X_train, X_test, y_train, y_test = tr_te_split(x, y, test_size=0.3)
XGBT_test = xgb.XGBRegressor(gamma = 0, learning_rate = 0.1, max_depth = 4, n_estimators = 100)
XGBT_test.fit(X_test, y_test)
print("The oos-r2 score is %0.4f." % XGBT_test.score(X_test, y_test))

The oos-r2 score is 0.3980.


In [9]:
#Random Forest 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split as tr_te_split

x = df_train
y = df['expected_use_time'].ravel()
X_train, X_test, y_train, y_test = tr_te_split(x, y, test_size=0.8)
tree2 = RandomForestRegressor()
param_grid = {"max_depth": [1, 3, 5, 7, 9],
              "ccp_alpha": np.arange(0, 0.01, 0.005).tolist(),
              "min_samples_split": [10, 20, 40],
              "min_samples_leaf": [20, 40, 60],
              "max_leaf_nodes": [5, 20, 80]}
grid_cv_tree2 = GridSearchCV(tree2, param_grid, cv = 5)
grid_cv_tree2.fit(X_train,y_train)

print("The best parameters are ",grid_cv_tree2.best_params_) 
print() 
print("The score is %0.4f." % grid_cv_tree2.best_score_)

The best parameters are  {'ccp_alpha': 0.005, 'max_depth': 9, 'max_leaf_nodes': 80, 'min_samples_leaf': 40, 'min_samples_split': 20}

The score is 0.3812.


In [10]:
#Compute OOS score of the best model so far -- Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split as tr_te_split
x = df_train
y = df['expected_use_time'].ravel()
X_train, X_test, y_train, y_test = tr_te_split(x, y, test_size=0.3)

tree_test = RandomForestRegressor(ccp_alpha=0.005,max_depth=9,max_leaf_nodes=80,min_samples_leaf=40,min_samples_split=20)
tree_test.fit(X_test, y_test)
print("The oos-r2 score is %0.4f." % tree_test.score(X_test, y_test))

The oos-r2 score is 0.3969.


In [8]:
#Try if clustering improves
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split as tr_te_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

x = df_train
y = df['expected_use_time'].ravel()
X_train, X_test, y_train, y_test = tr_te_split(x, y, test_size=0.8)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

kmeans = KMeans(n_clusters=3, random_state=666, max_iter=10000,init='k-means++').fit(X_train)
X_train.insert(0,'cluster_index',kmeans.labels_,True)

In [9]:
cluster_index_test = kmeans.fit_predict(X_test_std)
X_test.insert(0,'cluster_index',cluster_index_test,True)

pred_score = np.array([])
y_true = np.array([])

In [10]:
# First cross validate to obtain the best model on cluster 0
param_grid = {"ccp_alpha": [0, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1],
              'n_estimators': [50, 100, 150],
              'max_depth': [1, 3, 5, 7, 9],
             }

Forest = RandomForestRegressor()

gsearch = GridSearchCV(Forest, param_grid, cv=5 ,scoring='r2').fit(X_train[X_train['cluster_index']==0],y_train[X_train['cluster_index']==0])

# Retrain on the entire training set of cluster 0

model1e = RandomForestRegressor(max_depth = gsearch.best_params_['max_depth'], ccp_alpha = gsearch.best_params_['ccp_alpha'], n_estimators = gsearch.best_params_['n_estimators']).fit(X_train[X_train['cluster_index']==0],y_train[X_train['cluster_index']==0])

# Compute the predicted probability of positive outcome probability for the testing set of cluster 0
pred_score = np.append(pred_score,np.array(model1e.predict(X_test[X_test['cluster_index']==0])))
y_true = np.append(y_true,np.array(y_test[X_test['cluster_index']==0]))


In [11]:
# Cross validate to find the best random forest regressor for cluster 1
gsearch =GridSearchCV(Forest, param_grid, cv=5 ,scoring='r2').fit(X_train[X_train['cluster_index']==1],y_train[X_train['cluster_index']==1])

model1e=RandomForestRegressor(max_depth = gsearch.best_params_['max_depth'], ccp_alpha = gsearch.best_params_['ccp_alpha'], n_estimators = gsearch.best_params_['n_estimators']).fit(X_train[X_train['cluster_index']==1],y_train[X_train['cluster_index']==1])

# Compute the predicted probability of positive outcome probability for the testing set of cluster 1
pred_score = np.append(pred_score,np.array(model1e.predict(X_test[X_test['cluster_index']==1])))
y_true = np.append(y_true,np.array(y_test[X_test['cluster_index']==1]))

In [13]:
# Cross validate to find the best random forest regressor for cluster 2
gsearch =GridSearchCV(Forest, param_grid = param_grid, cv=5 ,scoring='r2').fit(X_train[X_train['cluster_index']==2],y_train[X_train['cluster_index']==2])

model1e=RandomForestRegressor(max_depth = gsearch.best_params_['max_depth'], ccp_alpha = gsearch.best_params_['ccp_alpha']).fit(X_train[X_train['cluster_index']==2],y_train[X_train['cluster_index']==2])

# Compute the predicted probability of positive outcome probability for the testing set of cluster 2
pred_score = np.append(pred_score,np.array(model1e.predict(X_test[X_test['cluster_index']==2])))
y_true = np.append(y_true,np.array(y_test[X_test['cluster_index']==2]))

In [14]:
from sklearn.metrics import r2_score
print('The out-of-sample R2 for the cluster-specific model is: %0.4f' % r2_score(y_true,pred_score))

The out-of-sample R2 for the cluster-specific model is: -0.1549


By comparison, we can see that clustering does not improve the overall performance. So we just use the simple random forest model.

In [15]:
df_test = pd.read_csv('dataframe_test.csv')
df_test = df_test[['speed', 'max_load', 'weather_grade', 'source_type', 'grid_distance', 
                     'urgency', 'hour']]
df_test = pd.get_dummies(df_test)
for elem in df_test['hour'].unique():
    df_test[str(elem)] = (df_test['hour'] == elem).astype(int)
df_test = df_test.where(df_test.notnull(), 18)
df_test = df_test.drop(columns = ['nan'])
df_test = df_test.drop(columns = ['source_type_2.10E+18'])
df_test['6'] = 0
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25468 entries, 0 to 25467
Data columns (total 30 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   speed                               25468 non-null  float64
 1   max_load                            25468 non-null  int64  
 2   grid_distance                       25468 non-null  float64
 3   urgency                             25468 non-null  int64  
 4   hour                                25468 non-null  float64
 5   weather_grade_Bad Weather           25468 non-null  uint8  
 6   weather_grade_Normal Weather        25468 non-null  uint8  
 7   weather_grade_Slightly Bad Weather  25468 non-null  uint8  
 8   weather_grade_Very Bad Weather      25468 non-null  uint8  
 9   source_type_ASSIGN                  25468 non-null  uint8  
 10  source_type_DELIVERY                25468 non-null  uint8  
 11  source_type_PICKUP                  25468

In [16]:
df_test = df_test[['speed', 'max_load','grid_distance', 'urgency', 'hour', 'weather_grade_Bad Weather','weather_grade_Normal Weather',
                 'weather_grade_Slightly Bad Weather', 'weather_grade_Very Bad Weather', 'source_type_ASSIGN', 'source_type_DELIVERY', 'source_type_PICKUP',
                  '11.0','12.0','13.0','14.0','15.0','16.0','17.0','18.0','19.0','20.0','21.0','10.0','9.0','22.0','23.0','7.0','8.0','6']]
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25468 entries, 0 to 25467
Data columns (total 30 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   speed                               25468 non-null  float64
 1   max_load                            25468 non-null  int64  
 2   grid_distance                       25468 non-null  float64
 3   urgency                             25468 non-null  int64  
 4   hour                                25468 non-null  float64
 5   weather_grade_Bad Weather           25468 non-null  uint8  
 6   weather_grade_Normal Weather        25468 non-null  uint8  
 7   weather_grade_Slightly Bad Weather  25468 non-null  uint8  
 8   weather_grade_Very Bad Weather      25468 non-null  uint8  
 9   source_type_ASSIGN                  25468 non-null  uint8  
 10  source_type_DELIVERY                25468 non-null  uint8  
 11  source_type_PICKUP                  25468

In [17]:
x = df_train
y = df['expected_use_time'].ravel()

forest = RandomForestRegressor(ccp_alpha=0.005, max_depth=9, max_leaf_nodes=80, min_samples_leaf=40, min_samples_split=20)
forest.fit(x,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.005, criterion='mse',
                      max_depth=9, max_features='auto', max_leaf_nodes=80,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=40,
                      min_samples_split=20, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [18]:
result = forest.predict(df_test)

In [19]:
import csv
f = open('Regression.csv','w',encoding='utf-8',newline='')
csv_writer = csv.writer(f)
csv_writer.writerow(['order',"expected_use_time"])

for i in range(25468):
    csv_writer.writerow([i,result[i]])
f.close()